In [5]:
ls

Exer3.1.ipynb                  TrainExer 3-2.pdf
Exer3.3 Transformation .ipynb  TrainExer 3-3.pdf
Exer_3.2.ipynb                 TrainExer3_3.xlsx
Test3.pdf                      test3.xlsx
TrainExer 3-1-corrected.txt


In [107]:
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices, dmatrix
import numpy as np

In [108]:
df = pd.read_excel("TrainExer3_3.xlsx")

In [109]:
df.head()

,Year,Index,Dividends,Riskfree,LogEqPrem,BookMarket,NTIS,DivPrice,EarnPrice,Inflation
0,1927,17.66,0.77,0.0317,0.280823,0.374689,0.073955,-3.132667,-2.766942,-0.021176
1,1928,24.35,0.85,0.0426,0.313824,0.259667,0.042639,-3.355051,-2.870448,-0.009793
2,1929,21.45,0.97,0.0303,-0.112428,0.338458,0.165576,-3.096184,-2.589490,0.001830
3,1930,15.34,0.98,0.0148,-0.288025,0.554745,0.131525,-2.750667,-2.760923,-0.060665
4,1931,8.12,0.82,0.0241,-0.563742,1.170732,-0.011032,-2.292781,-2.588626,-0.095711


In [159]:
df["Log_Index_diff"] = np.log(df.Index).diff()

In [161]:
y, X  = dmatrices("Log_Index_diff ~ BookMarket + np.square(BookMarket)" ,data= df)

In [162]:
mod = sm.OLS(y, X).fit()

In [163]:
print (mod.summary())

                            OLS Regression Results                            
Dep. Variable:         Log_Index_diff   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     5.050
Date:                Sat, 05 May 2018   Prob (F-statistic):            0.00852
Time:                        21:02:40   Log-Likelihood:                 22.599
No. Observations:                  86   AIC:                            -39.20
Df Residuals:                      83   BIC:                            -31.84
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.05

In [164]:
y, X2  = dmatrices("np.log(Index) ~ BookMarket" ,data= df)

In [165]:
mod2 = sm.OLS(y, X2).fit()

In [166]:
print (mod2.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(Index)   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     64.76
Date:                Sat, 05 May 2018   Prob (F-statistic):           4.54e-12
Time:                        21:03:37   Log-Likelihood:                -143.25
No. Observations:                  87   AIC:                             290.5
Df Residuals:                      85   BIC:                             295.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.0254      0.329     21.332      0.0

In [116]:
print (mod3.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(Index)   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     36.38
Date:                Sat, 05 May 2018   Prob (F-statistic):           4.08e-08
Time:                        17:06:00   Log-Likelihood:                -152.39
No. Observations:                  87   AIC:                             308.8
Df Residuals:                      85   BIC:                             313.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 5.64

In [117]:
df["change_year"] = np.where(df.Year >= 1980, "a1980", "b1980")

In [118]:
df_1980 = df.copy()

In [119]:
df_1980.head()

,Year,Index,Dividends,Riskfree,LogEqPrem,BookMarket,NTIS,DivPrice,EarnPrice,Inflation,change_year
0,1927,17.66,0.77,0.0317,0.280823,0.374689,0.073955,-3.132667,-2.766942,-0.021176,b1980
1,1928,24.35,0.85,0.0426,0.313824,0.259667,0.042639,-3.355051,-2.870448,-0.009793,b1980
2,1929,21.45,0.97,0.0303,-0.112428,0.338458,0.165576,-3.096184,-2.589490,0.001830,b1980
3,1930,15.34,0.98,0.0148,-0.288025,0.554745,0.131525,-2.750667,-2.760923,-0.060665,b1980
4,1931,8.12,0.82,0.0241,-0.563742,1.170732,-0.011032,-2.292781,-2.588626,-0.095711,b1980


In [120]:
df_1980["Log_Index_diff"] = np.log(df_1980.Index).diff() #pandas series diff

In [121]:
df_1980["BM_80"] = df_1980.BookMarket * np.where(df_1980["change_year"] == "b1980", 0, 1)

In [130]:
y,X3= dmatrices("Log_Index_diff ~ BookMarket + BM_80" , data = df_1980)

In [123]:
X4 = dmatrix("change_year", data = df_1980)

In [133]:
y, X4 = dmatrices("Log_Index_diff ~ BookMarket+ BookMarket:change_year", df_1980)

In [135]:
import numpy as np

class MyTreat(object):
    def __init__(self, reference=0):
        self.reference = reference

    def code_with_intercept(self, levels):
        return ContrastMatrix(np.eye(len(levels)),
                              ["[My.%s]" % (level,) for level in levels])

    def code_without_intercept(self, levels):
        eye = np.eye(len(levels) - 1)
        contrasts = np.vstack((eye[:self.reference, :],
                               np.zeros((1, len(levels) - 1)),
                               eye[self.reference:, :]))
        suffixes = ["[MyT.%s]" % (level,) for level in
                    levels[:self.reference] + levels[self.reference + 1:]]
        return ContrastMatrix(contrasts, suffixes)

In [154]:
l=["b1980", "a1980"]
dmatrix("C(change_year, levels = l) -1", df_1980)

DesignMatrix with shape (87, 2)
  C(change_year, levels=l)[b1980]  C(change_year, levels=l)[a1980]
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                                1                                0
                              

In [155]:
y, X5 = dmatrices("Log_Index_diff ~ BookMarket+ BookMarket:C(change_year, levels = l)", df_1980)

In [157]:
mod5 = sm.OLS(y, X5).fit()

In [158]:
print (mod5.summary())

                            OLS Regression Results                            
Dep. Variable:         Log_Index_diff   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     3.778
Date:                Sat, 05 May 2018   Prob (F-statistic):             0.0269
Time:                        17:18:47   Log-Likelihood:                 21.408
No. Observations:                  86   AIC:                            -36.82
Df Residuals:                      83   BIC:                            -29.45
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [80]:
mod_3 = sm.OLS(y, X3).fit()

In [82]:
print (mod_3.summary())

                            OLS Regression Results                            
Dep. Variable:         Log_Index_diff   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     3.778
Date:                Sat, 05 May 2018   Prob (F-statistic):             0.0269
Time:                        16:23:11   Log-Likelihood:                 21.408
No. Observations:                  86   AIC:                            -36.82
Df Residuals:                      83   BIC:                            -29.45
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1661      0.054      3.094      0.0

### coding categorical data with patsy


In [64]:
from patsy import dmatrix, demo_data, ContrastMatrix, Poly

In [72]:
demo_data("a", nlevels= 3)

{'a': ['a1', 'a2', 'a3', 'a1', 'a2', 'a3']}

In [87]:
df_test = pd.DataFrame(demo_data("a", nlevels= 3))

In [89]:
dmatrix("a", df_test)

DesignMatrix with shape (6, 3)
  Intercept  a[T.a2]  a[T.a3]
          1        0        0
          1        1        0
          1        0        1
          1        0        0
          1        1        0
          1        0        1
  Terms:
    'Intercept' (column 0)
    'a' (columns 1:3)